In [34]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pymongo

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

In [12]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [13]:
html = browser.html
news_soup = BeautifulSoup(html, "html.parser")
slide_element = news_soup.select_one("ul.item_list li.slide")

In [14]:
slide_element.find("div", class_="content_title")

<div class="content_title"><a href="/news/8678/the-detective-aboard-nasas-perseverance-rover/" target="_self">The Detective Aboard NASA's Perseverance Rover</a></div>

In [15]:
news_title = slide_element.find("div", class_="content_title").get_text()
print(news_title)

The Detective Aboard NASA's Perseverance Rover


In [16]:
news_paragraph = slide_element.find("div", class_="article_teaser_body").get_text()
print(news_paragraph)

An instrument called SHERLOC will, with the help of its partner WATSON, hunt for signs of ancient life by detecting organic molecules and minerals.


# JPL Mars Space Images - Featured Image

In [17]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [18]:
full_image_button = browser.find_by_id("full_image")
full_image_button.click()

In [19]:
browser.is_element_present_by_text("more info")
more_info_element = browser.links.find_by_partial_text("more info")
more_info_element.click()

In [20]:
html = browser.html
image_soup = BeautifulSoup(html, "html.parser")

In [21]:
img_url = image_soup.select_one("figure.lede a img").get("src")
img_url

'/spaceimages/images/largesize/PIA16239_hires.jpg'

In [23]:
img_url = f"https://www.jpl.nasa.gov{img_url}"
print(img_url)

https://www.jpl.nasa.govhttps://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16239_hires.jpg


# Mars Weather

In [24]:
url = "https://twitter.com/marswxreport?lang=en"
request_twitter = requests.get(url)

In [25]:
soup_twitter = BeautifulSoup(request_twitter.text, 'html.parser')

In [26]:
mars_weather_tweets = [p.text for p in soup_twitter.findAll('p', class_='tweet-text')]

In [27]:
mars_weather = mars_weather_tweets[0]
mars_weather

'InSight sol 533 (2020-05-27) low -92.8ºC (-135.1ºF) high -6.7ºC (19.9ºF)\nwinds from the SW at 6.4 m/s (14.2 mph) gusting to 21.3 m/s (47.5 mph)\npressure at 7.20 hPapic.twitter.com/sKQ2HEz7A2'

# Mars Facts

In [28]:
mars_df = pd.read_html("https://space-facts.com/mars/")[0]
mars_df.columns=["Description", "Value"]
mars_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


# Mars Hemispheres

In [29]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [30]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [31]:
mars_hemisphere = []

products = soup.find("div", class_ = "result-list" )
hemispheres = products.find_all("div", class_="item")

for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup=BeautifulSoup(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    mars_hemisphere.append({"title": title, "img_url": image_url})

In [32]:
mars_hemisphere

[{'title': 'Cerberus Hemisphere ',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere ',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere ',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere ',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]